# Robust methods for Machine Learning

## Let's start simple: attack a linear model

#### Tutorial #1 (Anne Gagneux)

In [ ]:
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

We are going to attack a linear model for binary classification.
We focus on MNIST dataset where we only keep the $3$ and $7$ digits.
In our setting, $\mathbf X_{\text{train}}$ is the training dataset of images ($7$ and $3$) and $y_{\text{train}}$ are the matching ground-truth labels.

Our linear model builds a decision function based on a hyperplane:
$$ y_{\text{pred}} = \text{sign} (w^T x + b) $$

The algorithm, i.e. Logistic regression, learns $w$ and $b$.

In [ ]:
# Load MNIST
mnist_train = datasets.MNIST(
    "./data", train=True, download=True, transform=transforms.ToTensor())
mnist_test = datasets.MNIST("./data", train=False,
                            download=True, transform=transforms.ToTensor())


# Only keep 3 and 7
train_idx = (mnist_train.targets == 3) + (mnist_train.targets == 7)

mnist_train.data = mnist_train.data[train_idx]
mnist_train.targets = mnist_train.targets[train_idx]

test_idx = (mnist_test.targets == 3) + (mnist_test.targets == 7)
mnist_test.data = mnist_test.data[test_idx]
mnist_test.targets = mnist_test.targets[test_idx]

x_train, y_train = mnist_train.data.numpy(), mnist_train.targets.numpy()
x_train = x_train.reshape(
    (x_train.shape[0], x_train.shape[1]*x_train.shape[2]))
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)

x_test, y_test = mnist_test.data.numpy(), mnist_test.targets.numpy()
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1]*x_test.shape[2]))
x_test = scaler.transform(x_test)

In [ ]:
# Train logistic regression
logreg = LogisticRegression(solver='lbfgs', max_iter=2000)
logreg.fit(x_train, y_train)
print('test score = {}'.format(logreg.score(x_test, y_test)))

In [ ]:
x1 = x_test[0]
# Pick a point in the dataset


def show(x, classifier):
    plt.title('Prediction: %s. Confidence: %d %%' %
              (classifier.predict([x])[0],
               100 * classifier.predict_proba([x]).max()),
              fontsize=18)
    xx = scaler.inverse_transform([x]).reshape((28, 28))
    plt.imshow(xx, cmap=plt.cm.gray_r, vmin=0, vmax=255)
    plt.axis('off')


show(x1, logreg)

![Projection](decision-boundary.jpg)

If we denote $x_1$ our image. 
The shorter distance to a point at the frontier is the orthogonal projection on the hyperplane $w^T x + b = 0$.

<span style="color:orange">**Write the projection operator onto the hyperplane**</span>

In [ ]:
w = logreg.coef_[0]
b = logreg.intercept_

x2 = # TO COMPLETE
show(x2, logreg)

What if we want to force prediction of $x_1$ a $3$ ? 

<span style="color:orange">**Write an explicit formula forcing $x_1$ to be misclassified**</span>


![Projection](decision-boundary-2.jpg)



In [ ]:
x3 = # TO COMPLETE
show(x3)

Up to now, we have minimized the $\ell_2$ distance.
Indeed, the orthogonal projection writes as:
$$\min_x \Vert x-x_1 \Vert_2 \text{ subject to } w^Tx+ b = 0$$

What if we want to minimize the maximum variation of each pixel ? 
$\rightarrow$ We use the $\ell_\infty$ distance.

Our new minimization problem is:
$$\min_x \Vert x-x_1 \Vert_\infty \text{ subject to } w^Tx+ b = 0$$

<span style="color:orange">**Solve the $\ell_\infty$ optimization problem**</span>

*Recall (Holder's Inequality)*
$$|x^T y| \leq \Vert x \Vert_1 \Vert y \Vert_\infty$$

![Projection infty](decision-boundary-infty.jpg)

In [ ]:
x2_infty = # TO COMPLETE
show(x2_infty, logreg)